Setting Kaggle by creating directory /root/.kaggle/

Creating file kaggle.json with username and key

Setting the rights of the file in order to be readable and writeable only by this user

(We know that this is not a safe way to do it because it shows username and key in clear, but those are accademic-purpose informations so we felt like show them)

In [ ]:
!mkdir /root/.kaggle/
with open('/root/.kaggle/kaggle.json', 'w') as writefile:
    writefile.write('{\"username\":\"albertoboggio\",\"key\":\"51842ae4d703c2aa0fd682ba4a620a79\"}')
!chmod 600 /root/.kaggle/kaggle.json

Downloading the dataset IMDB and decompressing the file title.principals.tsv

In [ ]:
!kaggle datasets download ashirwadsangwan/imdb-dataset --force
!unzip imdb-dataset.zip title.principals.tsv.gz
!gunzip -k title.principals.tsv.gz

100% 1.44G/1.44G [00:09<00:00, 137MB/s]
100% 1.44G/1.44G [00:09<00:00, 155MB/s]
Archive:  imdb-dataset.zip
replace title.principals.tsv.gz? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: title.principals.tsv.gz  
gzip: title.principals.tsv already exists; do you wish to overwrite (y or n)? ^C


This function reads the file containing the baskets and extract from it a list of baskets (represented by lists of items) where only actors appear as items.

In [ ]:
def load_baskets(f_name):
    """
        Take the f_name (filename) of the file in .tsv format
        and loads the baskets in main memory.
        Return a list of lists.
        Every list is a basket and contains the items sorted by id.
    """

    with open(f_name, mode="r", encoding='utf8') as f:

        baskets = list()
        curr_basket = '1'
        b = list() # Contains the items of the curr_basket

        f.readline()
        
        for row in f:
            row_data = row.strip().split('\t')
            basket = (row_data[0][2:]).lstrip('0')
            item = (row_data[2][2:]).lstrip('0')

            """
            We can do the following because we know that the datasets is sorted
            according to the movie index, so we know that whenever the movie 
            changes it will be a new movie not already processed
            """
            if(row_data[3] == "actor" or row_data[3] == "actress"):
                if(curr_basket != basket):
                    if(len(b) != 0):
                        baskets.append(sorted(b))
                    b = list()
                    curr_basket = basket

                b.append(int(item))

        baskets.append(sorted(b))

    return baskets

In the following code we calculate the number of:
1. items (I)
2. baskets (B)
3. possible pairs (P)
4. worst case of different pairs in baskets (WP)

In [ ]:
import scipy.special

baskets = load_baskets('title.principals.tsv')

I = len(set([item for b in baskets for item in b]))#number of items

B = len(baskets)#number of baskets

P = scipy.special.comb(I, 2)#number of possible pairs

WP = sum([scipy.special.comb(len(b), 2) for b in baskets])#worst case on the n. of different pairs in baskets

print(f"I={I}", f"B={B}", f"P={P}", f"WP={WP}")

I=1868056 B=3697162 P=1744815675540.0 WP=31901860.0


The following functions are useful for the implementation of the Apriori algorithm

In [ ]:
import itertools as it
import scipy.special

def pair_to_index(n, inverted=False):
    """
        n is the maximum number of indexes
    """
    def p_t_i(pair):
        if inverted:
            j, i = pair
        else:
            i, j = pair
            
        return int((n*(n-1)/2) - (n-i)*((n-i)-1)/2 + j - i - 1)
    
    return p_t_i
    
def index_to_pair(n):
    """
        n is the maximum number of indexes
    """
    def i_t_p(k):
        i = n - 2 - int((-8*k + 4*n*(n-1)-7)**(1/2)/2.0 - 0.5)
        j = k + i + 1 - n*(n-1)/2 + (n-i)*((n-i)-1)/2
        return (int(i), int(j))
    
    return i_t_p

def filt(cand_sets, supp_thr):
    """
        Return a set with the frequent subsets
        in cand_sets by checking whether their count is >=
        than the threshold.
    """
    return set([kset for kset, count in cand_sets.items() if count >= supp_thr])

def filt_pairs(cand_pairs, supp_thr, num_of_items):
    """
        Return the set of frequent pairs
        by checking whether their count is >=
        than the support threshold.
    """

    #Initialize the function for the conversion k->(i,j)
    i_t_p = index_to_pair(num_of_items)
    return set([i_t_p(index) for index, count in cand_pairs.items() if count >= supp_thr])

def pair_constr(freq_singles, baskets, num_of_items):
    """
        Takes the set of frequent singlentons, baskets
        and the max index of the items and returns a dictionary 
        with keys the candidate pairs and values their counts.
    """

    #Initialize the functions for the conversion (i,j)<->k
    p_t_i = pair_to_index(num_of_items)
    i_t_p = index_to_pair(num_of_items)
                          
    cand_pairs = dict()

    #Remove the not frequent singles from the baskets
    baskets[:] = [[item for item in b if (item,) in freq_singles] for b in baskets if len(b) > 0]

    #Construct the candidate pairs
    for b in baskets:
        for pair in it.combinations(b, 2):
            index = p_t_i(pair)
            if index not in cand_pairs:
                cand_pairs[index] = 1
            else:
                cand_pairs[index] += 1

    return cand_pairs

def tuple_in_basket(ktuple, basket):
    """
        Returns whether all the items in the
        k-tuple are in the basket or not.
    """
    for item in ktuple:
        if item not in basket:
            return False
    
    return True

def itemfilter1(baskets, freq_sets, k):
    """
        Takes the baskets, the frequent sets of the previous
        iteration and k (index of the current iteration) and 
        returns a new list of baskets by filtering out the not
        necessary items from the baskets.
        This is a relaxation of the standard filtering (itemfilter2).
    """

    # Find  the items which appear in at least k-1 (k-1)-tuples
    count_items = dict()
    good_items = set()

    for k1tuple in freq_sets:
        for item in k1tuple:
            if item not in count_items:
                count_items[item] = 1
            else:
                count_items[item] += 1

    for item in count_items:
        if count_items[item] >= k-1:
            good_items.add(item)

    del count_items
    gc.collect()

    #Update the baskets based on the freq_sets
    new_baskets = list()

    for b in baskets:
        new_b = list()

        #Use only the necessary items
        for item in b:

            if item in good_items:
                new_b.append(item)

        if len(new_b) > 0:
            new_baskets.append(new_b)
    
    return new_baskets

def itemfilter2(baskets, freq_sets, k):
    """
        Takes the baskets, the frequent sets of the previous
        iteration and k (index of the current iteration) and 
        returns a new list of baskets by filtering out the not
        necessary items from the baskets.
    """

    new_baskets = list()

    for b in baskets:
        new_b = list()

        #Chooses the filter method for the current basket    
        if len(freq_sets) < scipy.special.comb(len(b), k-1):
            #Starts from the items in b and delete them if the condition is not met
            for item in b:
                count = 0
                
                for k1tuple in freq_sets:
                    if item in k1tuple and tuple_in_basket(k1tuple, b):
                        count += 1
                if count >= k-1:
                    new_b.append(item)
            
        else:
            #Starts from the k-1tuples in the basket, count their items
            #if it's frequent and then delete the items
            count_items = dict()

            for k1tuple in it.combinations(b, k-1):
                if k1tuple in freq_sets:
                    for item in k1tuple:
                        if item not in count_items:
                            count_items[item] = 1
                        else:
                            count_items[item] += 1
            
            for item in b:
                if item in count_items:
                    if count_items[item] >= k-1:
                        new_b.append(item)
            
        if len(new_b) > 0:
            new_baskets.append(new_b)

    return new_baskets


def construct(freq_sets, baskets, k, itemfilter=1):
    """
        Take as input the list of k-1 frequent tuples
        and returns a dictionary with the
        candidate k tuples and their count.
        Works for k>=3.
        itemfilter goes from 0 to 2:
          0 = no filtering
          1 = relaxation of the standard filt.
          2 = standard filtering
    """

    cand_sets = dict()

    if itemfilter == 1:

        baskets[:] = itemfilter1(baskets, freq_sets, k)
    
    elif itemfilter == 2:
        
        baskets[:] = itemfilter2(baskets, freq_sets, k)

    for b in baskets:
        #Construct the candidate sets
        for kset in it.combinations(b, k):
            if kset not in cand_sets:
                cand_sets[kset] = 1
            else:
                cand_sets[kset] += 1

    return cand_sets

This is an implementation of Apriori algorithm having as input parameters:
1. the set of baskets (returned by the load_baskets function)
2. the support threshold
3. the minimal and the maximal cardinality of the frequent itemsets to be returned (optional)
4. the verbose flag which specifies the will to print computational resources' details (optional)
5. the aggressiveness of the item filtering (the one before the counting phase) (optional)

In [ ]:
import gc
import sys
import time

def apriori(baskets, supp_thr, min_card = 2, max_card = float('inf'), verbose = False, itemfilter=0):
    
    #List of frequent sets
    all_freq_sets = list()

    #Init. the table of the current iteration candidate sets
    #This will contain the couts of the candidate sets
    cand_sets = dict()

    t = time.time()
    #FIRST PASS
    #Count the singletons in every basket
    for b in baskets:
        for single in it.combinations(b, 1):
            if single not in cand_sets:
                cand_sets[single] = 1
            else:
                cand_sets[single] += 1

    if(verbose):
        print(f"Memory used to count singletons:", sys.getsizeof(cand_sets)/1000000,"MB")
        print("Number of candidate singletons:", len(cand_sets))

    #Filter out the not-frequent singletons
    freq_sets = filt(cand_sets, supp_thr)

    if(verbose):
        print("Number of frequent singletons:", len(freq_sets))
        print("Time to count singletons:", time.time() - t, "sec\n")

    #Deallocate the counts for singletons
    del cand_sets
    gc.collect()

    #Save the freq. singletons in the list to return if necessary
    if(min_card == 1):
        all_freq_sets.extend(freq_sets)

    #SECOND PASS
    #Calculate the num. of possible pairs
    num_of_items = max([max(b) for b in baskets])

    t = time.time()
    #Count the pairs in every basket
    cand_sets = pair_constr(freq_sets, baskets, num_of_items)

    if(verbose):
        print(f"Memory used to count pairs:", sys.getsizeof(cand_sets)/1000000,"MB")
        print("Number of candidate pairs:", len(cand_sets))

    #Filter out the not-frequent pairs
    freq_sets = filt_pairs(cand_sets, supp_thr, num_of_items)

    if verbose:
        print("Number of frequent pairs:", len(freq_sets))
        print("Time to count pairs:", time.time() - t, "sec\n")

    #Deallocate the counts for pairs
    del cand_sets
    gc.collect()
    
    #Save the freq. pairs in the list to return if necessary
    if(min_card <= 2 and 2 <= max_card):
        all_freq_sets.extend(freq_sets)

    #K-TH PASS with k>=3
    k = 3
    while len(freq_sets) != 0 and k <= max_card:
        t = time.time()

        cand_sets = construct(freq_sets, baskets, k, itemfilter=itemfilter)
        if(verbose):
            print(f"Memory used to count {k}sets:", sys.getsizeof(cand_sets)/1000000,"MB")
            print(f"Number of candidate {k}sets:", len(cand_sets))

        #Filter out the not-frequent k-sets
        freq_sets = filt(cand_sets, supp_thr)

        if verbose:
            print(f"Number of frequent {k}sets:", len(freq_sets))
            print(f"Time to count {k}sets:", time.time() - t,"sec\n")

        #Deallocate the counts for k-sets
        del cand_sets
        gc.collect()

        if(min_card <= k and len(freq_sets) != 0):
            all_freq_sets.extend(freq_sets)
        
        k += 1

    return all_freq_sets

This is an implementation of the PCY. The parameters are the same of Apriori algorithm but is required an additional parameter ("buckets") which specifies the number of buckets to use (cardinality of the image of the hash functions).

In [ ]:
def pcy(baskets, supp_thr, buckets, min_card = 2, max_card = float('inf'), verbose = False, itemfilter=1):

    #Calculate the num. of possible pairs
    num_of_items = max([max(b) for b in baskets])
    
    #List of frequent sets
    all_freq_sets = list()

    #This will contain the couts of the candidate sets
    cand_sets = dict()
    #These will contain the counts of the buckets
    cand_buck1 = dict()
    cand_buck2 = dict()

    #Functions used to calcute the 2 hash
    p_t_i = pair_to_index(num_of_items)
    p_t_i2 = pair_to_index(num_of_items, inverted=True)

    t = time.time()
    #FIRST PASS
    #Count the singletons in every basket
    for b in baskets:
        for single in it.combinations(b, 1):
            if single not in cand_sets:
                cand_sets[single] = 1
            else:
                cand_sets[single] += 1
        
        # Count the candidate buckets of pairs using p_t_i as hash function
        for pair in it.combinations(b, 2):
            buck1 = p_t_i(pair) % buckets
            buck2 = p_t_i2(pair) % buckets
            if buck1 not in cand_buck1:
                cand_buck1[buck1] = 1
            else:
                cand_buck1[buck1] += 1
            
            if buck2 not in cand_buck2:
                cand_buck2[buck2] = 1
            else:
                cand_buck2[buck2] += 1

    if verbose:
        print(f"Memory used to count singletons:", sys.getsizeof(cand_sets)/1000000,"MB")
        print(f"Memory used to count buckets 1:", sys.getsizeof(cand_buck1)/1000000,"MB")
        print(f"Memory used to count buckets 2:", sys.getsizeof(cand_buck2)/1000000,"MB")
        print("Number of candidate singletons:", len(cand_sets))

    #Initialization of the 2 bitmap
    pairbit1 = set()
    pairbit2 = set()

    for buck1 in cand_buck1:
        if cand_buck1[buck1] > supp_thr:
            pairbit1.add(buck1)

    for buck2 in cand_buck2:
        if cand_buck2[buck2] > supp_thr:
            pairbit2.add(buck2)
    
    #Filter out the not-frequent singletons
    freq_sets = filt(cand_sets, supp_thr)

    if verbose:
        print("Number of frequent singletons:", len(freq_sets))
        print("Number of frequent buckets 1:", len(pairbit1), "/", buckets)
        print("Number of frequent buckets 2:", len(pairbit2), "/", buckets)
        print("Time to count singletons:", time.time() - t, "sec\n")

    #Deallocate the counts for singletons and buckets
    del cand_sets
    del cand_buck1
    del cand_buck2
    gc.collect()

    #Save the freq. singletons in the list to return if necessary
    if(min_card == 1):
        all_freq_sets.extend(freq_sets)

    #SECOND PASS                     
    cand_pairs = dict()

    t = time.time()

    #Remove the not frequent singles from the basket
    baskets[:] = [[item for item in b if (item,) in freq_sets] for b in baskets]

    for b in baskets:
        #Construct the candidate sets
        for pair in it.combinations(b, 2):
            index = p_t_i(pair)
            buck1 = index % buckets
            buck2 = p_t_i2(pair) % buckets
            # Consider only the pairs in the frequent buckets
            if (buck1 in pairbit1) and (buck2 in pairbit2):
                if index not in cand_pairs:
                    cand_pairs[index] = 1
                else:
                    cand_pairs[index] += 1

    if verbose:
        print(f"Memory used to count pairs:", sys.getsizeof(cand_pairs)/1000000,"MB")
        print("Number of candidate pairs:", len(cand_pairs))

    #Deallocate the "bitmaps"
    del pairbit1
    del pairbit2
    gc.collect()

    #Filter out the not-frequent pairs
    freq_sets = filt_pairs(cand_pairs, supp_thr, num_of_items)

    if verbose:
        print("Number of frequent pairs:", len(freq_sets))
        print("Time to count pairs:", time.time() - t, "sec\n")

    #Deallocate the counts for pairs
    del cand_pairs
    gc.collect()
    
    #Save the freq. pairs in the list to return if necessary
    if(min_card <= 2 and 2 <= max_card):
        all_freq_sets.extend(freq_sets)

    #K>=3 PASSES
    k = 3
    while len(freq_sets) != 0 and k <= max_card:
        t = time.time()

        cand_sets = construct(freq_sets, baskets, k, itemfilter=itemfilter)

        if verbose:
            print(f"Memory used to count {k}sets:", sys.getsizeof(cand_sets)/1000000,"MB")
            print(f"Number of candidate {k}sets:", len(cand_sets))

        #Filter out the not-frequent k-sets
        freq_sets = filt(cand_sets, supp_thr)

        if verbose:
            print(f"Number of frequent {k}sets:", len(freq_sets))
            print(f"Time to count {k}sets:", time.time() - t,"sec\n")

        #Deallocate the counts for k-sets
        del cand_sets
        gc.collect()

        #print(f"Elapsed time on {k} iteration: ", time.time() - t)

        if(min_card <= k and len(freq_sets) != 0):
            all_freq_sets.extend(freq_sets)
        
        k += 1

    return all_freq_sets


Here, follows the implementation of a time-limited version of Apriori algorithm useful for the parameter tuning of the support threshold (which is next).

In [ ]:
import signal
import resource
import copy
from contextlib import contextmanager

@contextmanager
def timeout(time):
    signal.signal(signal.SIGALRM, raise_timeout)
    signal.alarm(time)

    try:
        yield
    except TimeoutError:
        pass
    finally:
        signal.signal(signal.SIGALRM, signal.SIG_IGN)

def raise_timeout(signum, frame):
    raise TimeoutError

def apriori_timeout(baskets, supp_thr, min_card = 2, max_card = float('inf'), expire = 300):
    with timeout(expire):
        return apriori(baskets, supp_thr, min_card, max_card)

    raise Exception("Timeout expired")

def support_tuning(baskets, t_limit, min_card = 2, max_card = float('inf'), precision = 0.01, verbose=False):
    left = 1
    right = len(baskets)

    while (right - left) / right > precision:
        m = (right + left) // 2

        if verbose:
            print("left =", left, "supp =", m, "right =", right)

        b = copy.deepcopy(baskets)
        try:
            t0 = time.time()
            apriori_timeout(b, m, expire=t_limit)
            t1 = time.time()
            if verbose:
                print("Elapsed", t1-t0, "seconds")

            right = m
        except:
            if verbose:
                print("Timeout")

            left = m
    
    return right

Here we defined a function to study the time consumption of the algorithms we presented

In [ ]:
import copy
import time

def time_complexity(baskets, thresholds, buckets=0, itemfilter=1):
    """
    time_complexity takes as parameters:
    1. the original dataset in baskets
    2. a list of support thresholds as thresholds
    3. the number of buckets for PCY (if not specified or zero, runs Apriori)
    4. the itemfiltering we want to use
    and prints the elapsed time within every computation of the desired
        algorithm for each support threshold
    """
    for s in thresholds:
        b = copy.deepcopy(baskets)
        if buckets == 0:
            t0 = time.time()
            apriori(b, s, itemfilter=itemfilter)
            t1 = time.time()

            print("Time elapsed using support threshold =", s, "with Apriori using item filter #", itemfilter,")", t1 - t0, "seconds")
        else:
            t0 = time.time()
            pcy(b, s, baskets=buckets, itemfilter=itemfilter)
            t1 = time.time()
            
            print("Time elapsed using support threshold =", s, "with Apriori using item filter #", itemfilter," and buckets", buckets,")", t1 - t0, "seconds")

        

Loads the dataset in the list of lists B

In [ ]:
B = load_baskets('title.principals.tsv')

Here, we use the verbose option on the 4 versions of the code we wanted to analyse.

In [ ]:
bask = copy.deepcopy(B)
F = apriori(bask, 300, verbose=True, itemfilter=0)

print("*****************************")

bask = copy.deepcopy(B)
F = apriori(bask, 300, verbose=True, itemfilter=1)

print("*****************************")

bask = copy.deepcopy(B)
F = apriori(bask, 300, verbose=True, itemfilter=2)

print("*****************************")

bask = copy.deepcopy(B)
F = pcy(bask, 300, 80000, verbose=True, itemfilter=2)

Memory used to count singletons: 83.886184 MB
Number of candidate singletons: 1868056
Number of frequent singletons: 6014
Time to count singletons: 10.211458921432495 sec

Memory used to count pairs: 5.242984 MB
Number of candidate pairs: 121151
Number of frequent pairs: 3892
Time to count pairs: 12.896682500839233 sec

Memory used to count 3sets: 10.485872 MB
Number of candidate 3sets: 262902
Number of frequent 3sets: 2773
Time to count 3sets: 3.5084221363067627 sec

Memory used to count 4sets: 10.485872 MB
Number of candidate 4sets: 319465
Number of frequent 4sets: 1774
Time to count 4sets: 3.279869794845581 sec

Memory used to count 5sets: 10.485872 MB
Number of candidate 5sets: 285349
Number of frequent 5sets: 1066
Time to count 5sets: 2.718259334564209 sec

Memory used to count 6sets: 10.485872 MB
Number of candidate 6sets: 198122
Number of frequent 6sets: 541
Time to count 6sets: 2.047327995300293 sec

Memory used to count 7sets: 5.242984 MB
Number of candidate 7sets: 103075
Numb

Here, we compute the time consumption relative to different values of support threshold using the Apriori algorithm without any item filter

In [ ]:
time_complexity(B, [4000, 3000, 2000, 1500, 1000, 700, 500, 300])

Time elapsed using support threshold = 4000 with Apriori using item filter # 1 ) 19.45033836364746 seconds
Time elapsed using support threshold = 3000 with Apriori using item filter # 1 ) 25.00787878036499 seconds
Time elapsed using support threshold = 2000 with Apriori using item filter # 1 ) 27.495540380477905 seconds
Time elapsed using support threshold = 1500 with Apriori using item filter # 1 ) 30.509689569473267 seconds
Time elapsed using support threshold = 1000 with Apriori using item filter # 1 ) 31.70386552810669 seconds
Time elapsed using support threshold = 700 with Apriori using item filter # 1 ) 35.450013160705566 seconds
Time elapsed using support threshold = 500 with Apriori using item filter # 1 ) 44.35437846183777 seconds
Time elapsed using support threshold = 300 with Apriori using item filter # 1 ) 49.795989751815796 seconds


In [ ]:
print("Support threshold obtained with 20 seconds:", support_tuning(B, 20))
print("Support threshold obtained with 25 seconds:", support_tuning(B, 25))
print("Support threshold obtained with 30 seconds:", support_tuning(B, 30))
print("Support threshold obtained with 35 seconds:", support_tuning(B, 35))
print("Support threshold obtained with 40 seconds:", support_tuning(B, 40))
print("Support threshold obtained with 45 seconds:", support_tuning(B, 45))
print("Support threshold obtained with 50 seconds:", support_tuning(B, 50))
print("Support threshold obtained with 55 seconds:", support_tuning(B, 55))

Support threshold obtained with 20 seconds: 931512
Support threshold obtained with 25 seconds: 2369
Support threshold obtained with 30 seconds: 1749
Support threshold obtained with 35 seconds: 804
Support threshold obtained with 40 seconds: 564
Support threshold obtained with 45 seconds: 504
Support threshold obtained with 50 seconds: 490


This is a basic implementation of the SON algorithm having as input parameters:
1. the list of the baskets
2. the fraction p of baskets to have in each chunk
3. the support threshold
4. the minimal and the maximal cardinality of the frequent itemsets to be returned (optional)

The main phases are the following:
1. it splits the baskets in chunks according to the passed fraction p
2. runs the apriori algorithm (alternatively we can use the PCY) on each chunk
with support threshold p*supp_thr and gets the candidate frequent itemsets
3. filters out the false positive itemsets by scanning all the baskets


In [ ]:
import math
import copy

def son(baskets, p, supp_thr, min_card = 2, max_card = float('inf')):
    
    #Calculates the chunk length and the number of chunks
    chunk_len = int(math.ceil(len(baskets)*p))
    num_chunks = int(math.ceil(len(baskets)/chunk_len))
    
    #Set of the candidate frequent itemsets
    cand_itemsets = set()
    
    #Runs the passed algorithm on all the chunks and gets
    #the set of candidate frequent itemsets
    for i in range(num_chunks):
        cand_itemsets.update(apriori(copy.deepcopy(baskets[chunk_len*i:chunk_len*(i+1)]), supp_thr*p, min_card, max_card))    

    #Dictionary to count the occurrences of each candidate 
    #frequent itemset on the set of baskets
    count = dict([(e,0)for e in cand_itemsets])

    #Counts their occurrencies    
    for b in baskets:
        for kset in cand_itemsets:
            if set(kset).issubset(set(b)):
                count[kset] += 1

    #Return only the itemsets with count >= than the support threshold
    return [kset for kset, count in count.items() if count >= supp_thr]

Here, we set up Spark and Hadooop.

In [ ]:
#Setting up spark and hadoop
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://www-eu.apache.org/dist/spark/spark-3.1.2/spark-3.1.2-bin-hadoop2.7.tgz
!tar xf spark-3.1.2-bin-hadoop2.7.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.2-bin-hadoop2.7"

import findspark
findspark.init("spark-3.1.2-bin-hadoop2.7")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

import pyspark

This is the implementation of the SON algorithm by using 
the Map-Reduce framework with Spark and Hadoop.

The main phases are the following:
1. First map function: takes the chunk and returns the candidate frequent itemsets by using the A-Priori algorithm with support threshold p*supp_thr. The output is a set of key value pairs (kset, 1) where kset is a frequent itemset from the chunk.
2. First reduce function: simply ignores the value and produce the key of the itemset. The output is the set of candidate frequent itemsets.
3. Second map function: takes the set of candidate frequent itemsets and a chunk and returns the number of occurrences of each of the candidate itemsets among the baskets in the chunk. The output is a set of key value pairs (kset, count) where kset is the candidate frequent itemset and count is the number of occurrencies in the chunk.
4. Second reduce function: returns the sum of the occurrencies of a given itemset.
5. Those itemsets whose count on all the baskets is >= than the support threshold are returned as frequent itemsets (filtering of false positives).

In [ ]:
import math

def count_sets(chunk, sets):
    """
        Function that counts the number of occurrencies
        of the itemsets in the baskets contained in the chunk.
        
        chunk: list of baskets
        sets: list of candidate frequent itemsets

        return: a list with pairs (kset, count) indicating
        the number of occurrencies of each itemset in the chunk
    """
    counts = dict([(kset, 0)for kset, value in sets])
       
    for b in chunk:
        for kset, value in sets:
            if set(kset).issubset(set(b)):
                counts[kset] += 1

    return [(kset, count) for kset, count in counts.items()]

def map_reduce_son(baskets, p, supp_thr, min_card = 2, max_card = float('inf'), algo = apriori):

    sc = spark.sparkContext

    chunk_len = int(math.ceil(len(baskets)*p))
    num_chunks = int(math.ceil(len(baskets)/chunk_len))
    
    rdd = sc.parallelize([baskets[chunk_len*i:chunk_len*(i+1)] for i in range(num_chunks)])
    
    #First map-reduce step
    candidate_sets = (rdd.flatMap(lambda chunk: algo(chunk, supp_thr*p, min_card, max_card))
               .map(lambda kset: (kset, 1))
               .reduceByKey(lambda a, b: a)
               .collect())
    
    #Second map-reduce step
    out = (rdd.flatMap(lambda chunk: count_sets(chunk, candidate_sets))
               .reduceByKey(lambda a, b: a+b)
               .collect())
    
    #Filter out the false positives and return the freq. itemsets
    return [kset for kset, count in out if count >= 3000]

Here we check the correctness of the algorithm by comparing the results with the standard APriori.

In [ ]:
Baskets = copy.deepcopy(B)

print(map_reduce_son(B, 0.3, 3000))
print(apriori(Baskets, 3000))